### DECISION TREES PART I
Jakub Prusak 151178
<br> Katarzyna Przepiórska 151752

#### 1. Wstęp
Zadanie polega na stworzeniu drzewa decyzyjnego na podstawie danych zawierających informacje o podróżujących Tytanikiem.

#### 2. Wczytanie danych z pliku
Pierwszym krokiem jest wczytanie danych z pliku .csv i zapisanie ich jako wartości w słowniku ``passengers``.

In [191]:
import csv
import math

passengers = {
    "id" : [],
    "p_class" : [],
    "name" : [],
    "sex" : [],
    "age" : [],
    "siblings" : [],
    "children" : [],
    "survived" : []
}

with open("titanic-homework.csv", "r") as file:
    titanicFile = csv.DictReader(file)
    titanicList = list(titanicFile)
    print(titanicList)
    print(2)
    for line in titanicList:
        passengers["id"].append(line['PassengerId'])
        passengers["p_class"].append(line['Pclass'])
        passengers["name"].append(line['Name'])
        passengers["sex"].append(line['Sex'])
        passengers["age"].append(line['Age'])
        passengers["siblings"].append(line['SibSp'])
        passengers["children"].append(line['Parch'])
        passengers["survived"].append(line['Survived'])

print (passengers)

#### 3. Obliczanie prawdopodobieństwa
Przed przystąpieniem do następnego kroku, utworzona została funkcja ``get_probability()`` obliczająca prawdopodobieństwo przeżycia lub nieprzeżycia dla każdej wartości klasy. Funkcja zwraca słownik, w którym kluczami są nazwy klas, a wartościami listy. Pola w każdej z takich list oznaczają kolejno: prawdopodobieństwo przeżycia, prawdopodobieństwo nieprzeżycia oraz stosunek liczności danej klasy do wszystkich pasażerów (wartość ta będzie wykorzystana w obliczaniu entropii warunkowej).

In [192]:
def get_probability(passengers,key):
    new_dict  = {}
    values = []
    i = 0
    probability_dict = {}  
    
    
    if key == 'age':
        new_dict  = {}
        for age in passengers[key]:
            if(int(age)<=20):
                x='young'
            elif(int(age)<=40):
                x='middle'
            else:
                x='old'
            if x not in values:
                values.append(x)
                new_dict[x] = [0,0]
            if(passengers["survived"][i] == '1'):
                new_dict[x][1] += 1
            else:
                new_dict[x][0] += 1
            i+=1
        for field in new_dict:
            probability_dict[field]=[new_dict[field][0]/(new_dict[field][0] + new_dict[field][1]),new_dict[field][1]/(new_dict[field][0] + new_dict[field][1])]
            probability_dict[field].append((new_dict[field][0] + new_dict[field][1])/i)
    else:    
        for x in passengers[key]:
            if x not in values:
                values.append(x)
                new_dict[x] = [0,0]
            if(passengers["survived"][i] == '1'):
                new_dict[x][1] += 1
            else:
                new_dict[x][0] += 1
            i+=1
        for field in new_dict:
            probability_dict[field]=[new_dict[field][0]/(new_dict[field][0] + new_dict[field][1]),new_dict[field][1]/(new_dict[field][0] + new_dict[field][1])]
            probability_dict[field].append((new_dict[field][0] + new_dict[field][1])/i)
        
    return probability_dict

#### 4. Entropia, entropia warunkowa i information gain
Aby podjąć decyzję dotyczącą wyboru następnego węzła, wybrać należy ten atrybut, który ma największą wartość **information gain**, czyli różnicy **entropii** i **entropii warunkowej**.
W tym celu utworzono dwie funkcje - ``entropy()`` i ``conditional_entropy()`` wykorzystywane później do obliczenia wartości information gain, która zapisywana jest w słowniku o tej samej nazwie. W funkcji ``choose_leaf()`` wybierany jest następny liść drzewa na podstawie największej wartości innformation gain. Wykorzystano przedstawione na zajciach wzory:
$$
Entropy(S) = -\sum_{i=1}^n P(x_{i})logP(x_{i})
$$
$$
Entropy(S|A) = \sum_{j=1}^m \frac{|S_{j}|}{|S|}Entropy(S_{j})
$$
$$
Gain(S,A) = Entropy(S) − Entropy(S|A)
$$

In [193]:
probabilities = {}
probabilities['p_class'] = get_probability(passengers, 'p_class')
probabilities['sex'] = get_probability(passengers, 'sex')
probabilities['siblings'] = get_probability(passengers, 'siblings')
probabilities['children'] = get_probability(passengers, 'children')
probabilities['age'] = get_probability(passengers, 'age')

def entropy(lst):
    p = 0
    for x in lst:
        if x:
           p -= x * math.log2(x)
    return p

def conditional_entropy(probabilities, key):
    e_sum = 0
    for un_key in probabilities[key]:
        e_sum += probabilities[key][un_key][-1] * entropy(probabilities[key][un_key][:-1])
    return e_sum


def choose_leaf(probabilities):
    information_gain = {}
    max_ratio = 0
    max_ratio_key = ""
    for key in probabilities:
        information_gain[key] = 1 - conditional_entropy(probabilities, key)
        if information_gain[key] > max_ratio:
            max_ratio = information_gain[key]
            max_ratio_key = key
    return max_ratio, max_ratio_key, information_gain

choose_leaf(probabilities)
        

#### 5. Intrinsic information i gain ratio
W przypadku dużej ilości różych wartości można wykorzystać wartość **gain ratio**, czyli wartość information gain znormalzowaną z użyciem **intrinsic information**, które pod uwagę bierze wielkość poszczególnych gałęzi. Gain ratio rozpatruje nie tylko czystość węzłów, ale także rozłożenie przykładów poniędzy gałęzie drzewa. W celu obliczenia obu wartości zaimplementowano funkcje ``intrinsic_info()`` oraz ``gain_ratio()``. Do wykonania obu obliczeń wykorzystane zostały następujące wzory:
$$
IntrinsicInfo(S,A) = -\sum_{i=1}^n \frac{|S_{j}|}{|S|}log_{2}\frac{|S_{j}|}{|S|}
$$
$$
GainRatio(S,A) = \frac{Gain(S,A)}{IntrinsicInfo(S,A)}
$$

In [194]:
def intrinsic_info(probabilities, key):
    e_sum = 0
    for un_key in probabilities[key]:
        e_sum -= probabilities[key][un_key][-1] * math.log2(probabilities[key][un_key][-1])
    return e_sum

def gain_ratio(probabilities):
    gain_ratio ={}
    for key in probabilities:
        gain_ratio[key] = choose_leaf(probabilities)[2][key]/(intrinsic_info(probabilities, key)) 
    return gain_ratio

gain_ratio(probabilities)


#### 6. Utworzenie gałęzi drzewa i wyświetlanie drzewa

In [196]:
def split_by_key(passengers,key):
    values = []
    res = {key:{}}
    i = 0    
    if key == 'age':
        res[key]  = {}
        for age in passengers[key]:
            if(int(age)<=20):
                x='young'
            elif(int(age)<=40):
                x='middle'
            else:
                x='old'
            if x not in values:
                values.append(x)
                res[key][x] = {}
            for field in passengers:
                if field not in res[key][x]:
                    res[key][x][field]=[]
                res[key][x][field].append(passengers[field][i])
            i+=1
    else:    
        for k in passengers[key]:
            if k not in values:
                values.append(k)
                res[key][k]={}
            for field in passengers:
                if field not in res[key][k]:
                    res[key][k][field]=[]
                res[key][k][field].append(passengers[field][i])
            i+=1
            
    return res
        
def print_tree(root:dict,cd,md):
    if(root == "UwU"):
        return
    if("survived" in root):
        if((root["survived"].count("1") == len(root["survived"])) or (root["survived"].count("0") == len(root["survived"]))):
            print(' '*(cd-1) +chr(9492)+ "survived")
            if(root["survived"][0]=="1"):
                print(' '*cd +chr(9492)+ "yes")
                return
            print(' '*cd +chr(9492)+ "no")
            
    for key in root:
        if(key=="id"):
            return
        if(cd != 0):
            print(' '*(cd-1) +chr(9492)+ key)
        else:
            print(key)
        print_tree(root[key],cd+1,md)
        
def build_tree(root,cd=0,md=4):
    if("survived" in root):
        if((root["survived"].count("1") == len(root["survived"])) or (root["survived"].count("0") == len(root["survived"]))):
            return(root)
            
    probabilities = {}
    probabilities['p_class'] = get_probability(root, 'p_class')
    probabilities['sex'] = get_probability(root, 'sex')
    probabilities['siblings'] = get_probability(root, 'siblings')
    probabilities['children'] = get_probability(root, 'children')
    probabilities['age'] = get_probability(root, 'age')
    tree = {}
    
    max_ratio,best_key,_ = choose_leaf(probabilities)
    if(max_ratio==1):
        return(root)
    tree = split_by_key(root,best_key)
    
    for key in tree[best_key]:
        node = build_tree(tree[best_key][key],cd+1)
        if node != "done":
            tree[best_key][key]=node
    
    return tree

tree = build_tree(passengers)
print_tree(tree,0,3)


sex
└male
 └p_class
  └3
   └children
    └0
     └survived
      └no
    └1
    └5
     └survived
      └no
    └2
     └survived
      └no
    └3
     └survived
      └no
  └1
   └age
    └old
     └survived
      └no
    └middle
    └young
  └2
└female
 └siblings
  └1
   └p_class
    └1
     └survived
      └yes
    └2
     └survived
      └yes
    └3
  └0
   └survived
    └yes
  └3
  └2
   └survived
    └no
  └4
   └survived
    └yes
  └5
   └survived
    └no
